# **Catégorisez automatiquement des questions**

## partie 3/8 : Prédiction de tags, approche non-supervisée

### <br> Proposition de mots clés, de type LDA avec visualisation en 2D des topics

<br>


## Importation des librairies, réglages


In [14]:
import os, sys, random
import ast
# from zipfile import ZipFile
import numpy as np
import pandas as pd
from pandarallel import pandarallel

# Visualisation
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px

# Feature engineering
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import Sparse2Corpus
from gensim.models import LdaModel, CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


print('\nPython version ' + sys.version)
print('pyLDAvis version ' + pyLDAvis.__version__)

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)



Python version 3.11.4 (main, Jul  5 2023, 14:15:25) [GCC 11.2.0]
pyLDAvis version 3.4.0

Number of CPU cores: 8
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Fonctions


In [2]:
def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))



### import


In [3]:
# import

train = pd.read_csv('./../data/cleaned_data/train_bow_uniques.csv', sep=',')
test = pd.read_csv('./../data/cleaned_data/test_bow_uniques.csv', sep=',')

quick_look(train)


shape : (42898, 8)


,CreationDate,title,body,all_tags,title_nltk,body_nltk,title_spacy,body_spacy
0,2019-06-05 15:13:02,How to use memset while handling strings in C++?,I am from Python background and recently learn...,"['c++', 'initialization', 'c-strings', 'string...","['memset', 'handle', 'string']","['memset', 'handle', 'string', 'python', 'back...","['use', 'memset', 'handle', 'string']","['background', 'learn', 'function', 'memset', ..."
1,2018-10-31 12:35:02,How to correct spelling in google docs using k...,I would like to be able to replace a misspelle...,"['gmail', 'keyboard-shortcuts', 'google-docs',...","['correct', 'spell', 'google', 'doc', 'keyboar...","['correct', 'spell', 'google', 'doc', 'shortcu...","['correct', 'spelling', 'keyboard', 'shortcut']","['like', 'replace', 'word', 'recommend', 'corr..."
2,2020-09-19 10:40:23,live server vscode on another computer,I have 2 computers. when I open the project wi...,"['visual-studio-code', 'server', 'localhost', ...","['server', 'vscode', 'computer']","['server', 'vscode', 'computer', 'open', 'proj...","['server', 'vscode', 'computer']","['computer', 'open', 'project', 'server', 'url..."
3,2012-10-23 16:47:04,django ajax post 403 forbidden,using django 1.4 im getting a 403 error when i...,"['javascript', 'ajax', 'django', 'http-post', ...","['django', 'ajax', 'forbidden']","['django', 'ajax', 'get', 'error', 'try', 'pos...",['forbid'],"['django', 'error', 'try', 'post', 'javascript..."
4,2019-04-21 16:10:24,Listen to changes and reload container on code...,I am using docker-compose in visual studio 201...,"['angular', 'visual-studio', 'docker', 'docker...","['listen', 'change', 'reload', 'container', 'c...","['listen', 'change', 'reload', 'container', 'c...","['listen', 'change', 'reload', 'container', 'c...","['docker', 'compose', 'studio', 'window', 'run..."


,CreationDate,title,body,all_tags,title_nltk,body_nltk,title_spacy,body_spacy
42893,2017-02-23 11:34:31,Do we need clear MDC after HTTP request in Spring,According to this answer thread local variable...,"['java', 'spring', 'logging', 'log4j', 'logback']","['need', 'mdc', 'request', 'spring']","['need', 'mdc', 'request', 'spring', 'accord',...","['need', 'request']","['accord', 'answer', 'thread', 'variable', 'us..."
42894,2011-10-13 20:57:32,How to make i18n with Handlebars.js (mustache ...,I'm currently using Handlebars.js (associated ...,"['javascript', 'jquery', 'internationalization...","['make', 'i18n', 'handlebar', 'template']","['make', 'i18n', 'handlebar', 'template', 'ass...",['template'],"['associate', 'web', 'app', 'client', 'render'..."
42895,2012-09-06 00:16:46,How can I make R read my environmental variables?,I am running R on EC2 spot instances and I nee...,"['linux', 'r', 'ubuntu', 'amazon-ec2', 'enviro...","['make', 'read', 'variable']","['make', 'read', 'variable', 'run', 'spot', 'i...","['read', 'variable']","['run', 'spot', 'instance', 'need', 'terminate..."
42896,2021-03-23 03:50:50,How to prevent react-query from fetching initi...,I'm using react-query v3.13 to fetch data from...,"['javascript', 'reactjs', 'fetch', 'react-quer...","['prevent', 'query', 'fetch', 'enable']","['prevent', 'query', 'fetch', 'enable', 'data'...","['prevent', 'react', 'query', 'fetch', 'enable']","['react', 'query', 'fetch', 'datum', 'want', '..."
42897,2016-03-17 04:19:15,Inserting into table with an Identity column w...,I have a table A_tbl in my database. I have cr...,"['sql', 'sql-server', 'database', 'ssms', 'dat...","['insert', 'table', 'identity', 'column', 'rep...","['insert', 'table', 'identity', 'column', 'rep...","['insert', 'table', 'column', 'replication', '...","['table', 'database', 'create', 'trigger', 'ca..."


uniques :


CreationDate    42893
title           42897
body            42898
all_tags        41513
title_nltk      42171
body_nltk       42898
title_spacy     37346
body_spacy      42891
dtype: int64

Doublons ?  0 



,column_name,missing,present,percent_missing,type
CreationDate,CreationDate,0,42898,0.0,object
title,title,0,42898,0.0,object
body,body,0,42898,0.0,object
all_tags,all_tags,0,42898,0.0,object
title_nltk,title_nltk,0,42898,0.0,object
body_nltk,body_nltk,0,42898,0.0,object
title_spacy,title_spacy,0,42898,0.0,object
body_spacy,body_spacy,0,42898,0.0,object


In [4]:
# Tt semble OK, sauf les types : le fait d'exporter nos données a converti nos listes de tokens en str

def turn_str_back_into_list(df):
    """Correct the type change due to .csv export"""

    df['title_nltk'] = df['title_nltk'].apply(ast.literal_eval)
    df['body_nltk'] = df['body_nltk'].apply(ast.literal_eval)
    df['title_spacy'] = df['title_spacy'].apply(ast.literal_eval)
    df['body_spacy'] = df['body_spacy'].apply(ast.literal_eval)


turn_str_back_into_list(train)
turn_str_back_into_list(test)


In [5]:
# Vérif
train[['title_nltk', 'body_nltk', 'title_spacy', 'body_spacy']].map(len).describe()

# OK, pas de liste vide (min = 1)


,title_nltk,body_nltk,title_spacy,body_spacy
count,42898.000000,42898.000000,42898.000000,42898.000000
mean,4.409903,39.608653,3.311413,29.030444
std,1.732934,27.997950,1.619366,20.274162
min,1.000000,2.000000,1.000000,1.000000
25%,3.000000,21.000000,2.000000,16.000000
50%,4.000000,32.000000,3.000000,24.000000
75%,5.000000,50.000000,4.000000,37.000000
max,14.000000,368.000000,12.000000,307.000000


In [7]:
# quick_look(test)


In [8]:
test[['title_nltk', 'body_nltk', 'title_spacy', 'body_spacy']].map(len).describe()
# OK


,title_nltk,body_nltk,title_spacy,body_spacy
count,4767.000000,4767.000000,4767.000000,4767.000000
mean,4.395427,39.790854,3.284665,29.205160
std,1.716085,27.418228,1.620959,19.973006
min,1.000000,3.000000,1.000000,1.000000
25%,3.000000,21.000000,2.000000,16.000000
50%,4.000000,32.000000,3.000000,24.000000
75%,5.000000,51.000000,4.000000,37.000000
max,13.000000,268.000000,12.000000,194.000000


In [9]:
# Utile si nos inputs st sous forme de string
# mais il semble qu'on va plutôt conserver la liste de tokens au final

def fix_false_null_values(df):
    """
    Replace NaN values in specified columns with the string 'null'.
    ! USE ONLY AFTER VERIFYING that the NaN values are all the "null" string
    """
    df.loc[(df['title_nltk'].isna()), 'title_nltk'] = 'null'
    df.loc[(df['title_spacy'].isna()), 'title_spacy'] = 'null'


# fix_false_null_values(train)
# fix_false_null_values(test)

# Check for null values in the entire DataFrame
# null_values = train[train.isnull().any(axis=1)]

# Print the rows with null values
# print(null_values)


In [10]:
# quick_look(train)
# quick_look(test)


In [11]:
index = [4532, 8280, 12992, 14957, 22934, 24964, 25950]

display(train.loc[train.index.isin(index), :])

# OK


,CreationDate,title,body,all_tags,title_nltk,body_nltk,title_spacy,body_spacy
4532,2013-10-23 22:23:31,How to change type of id in Microsoft.AspNet.I...,"(ASP.NET MVC 5, EF6, VS2013)\nI'm trying to fi...","['asp.net-mvc', 'entity-framework', 'asp.net-m...","[change, type, aspnet, identity, entityframewo...","[change, type, aspnet, identity, entityframewo...","[change, type, identity]","[try, figure, change, type, field, string, int..."
8280,2014-06-20 18:46:09,"Bootstrap form input: prevent submit, but allo...",I've got the following problem: \nI use bootst...,"['javascript', 'jquery', 'html', 'forms', 'twi...","[bootstrap, form, input, submit, allow, checking]","[bootstrap, form, input, submit, allow, check,...","[bootstrap, form, input, prevent, submit, allo...","[problem, use, bootstrap, form, input, user, p..."
12992,2017-08-21 19:46:31,PySpark: org.apache.spark.sql.AnalysisExceptio...,"I'm trying to load Parquet data into PySpark, ...","['python', 'apache-spark', 'pyspark', 'apache-...","[pyspark, apache, spark, attribute, name, cont...","[pyspark, apache, spark, attribute, name, cont...","[org.apache.spark.sql, analysisexception, cont...","[try, load, datum, column, space, aliase, erro..."
14957,2018-08-08 12:58:34,How to break ForEach Loop in TypeScript,"I have a the below code, on which i am unable ...","['javascript', 'angular', 'typescript', 'forea...","[break, loop, typescript]","[break, loop, typescript, code, condition, fun...",[break],"[code, break, loop, condition, function, let, ..."
22934,2014-11-26 18:26:05,Python: Creating a 2D histogram from a numpy m...,"I'm new to python.\nI have a numpy matrix, of ...","['python', 'numpy', 'matrix', 'matplotlib', 'h...","[python, create, histogram, matrix]","[python, create, histogram, matrix, dimension,...","[python, create, histogram, matrix]","[python, matrix, dimension, value, range, want..."
24964,2011-11-28 02:41:21,SSRS line chart not connecting data points,I've looked high and low and can't seem to fin...,"['join', 'reporting-services', 'graph', 'chart...","[line, chart, connect, data, point]","[line, chart, connect, data, point, look, seem...","[line, chart, connect, datum, point]","[look, find, answer, appear, issue, think, lin..."
25950,2014-08-21 15:58:49,GS1 barcode parsing,We need to parse the GS1 datamatrix barcode wh...,"['parsing', 'barcode', 'datamatrix', 'gs1-data...","[barcode, parse]","[barcode, parse, need, provide, party, know, l...","[barcode, parse]","[need, parse, barcode, provide, party, know, u..."


## LDA


In [16]:
# add random state
# add grid search cv
# add other score ? umap? c_v ? ...

def suggest_topics_using_LDA(df, feature):
    documents = df[feature].tolist()
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Set training parameters.
    num_topics = 10
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    temp = gensim_dictionary[0]  # This is only to "load" the dictionary.
    id2word = gensim_dictionary.id2token

    model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

    top_topics = model.top_topics(corpus, topn=20)

    # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
    # = umass ?
    avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
    print('Average topic coherence: %.4f.' % avg_topic_coherence)

    # Compute Coherence Score (cv)
    coherence_cv = CoherenceModel(model=model, texts=documents, dictionary=gensim_dictionary, coherence='c_v')
    coherence_lda_cv = coherence_cv.get_coherence()
    print('Coherence Score: ', coherence_lda_cv)

    # Compute Coherence Score (Umass)
    coherence_umass = CoherenceModel(model=model, texts=documents, dictionary=gensim_dictionary, coherence='u_mass')
    coherence_lda_umass = coherence_umass.get_coherence()
    print('u_mass Coherence Score: ', coherence_lda_umass)

    # Compute Coherence Score (npmi)
    coherence_npmi = CoherenceModel(model=model, texts=documents, dictionary=gensim_dictionary, coherence='c_npmi')
    coherence_lda_npmi = coherence_npmi.get_coherence()
    print('c_npmi Coherence Score: ', coherence_lda_npmi)

    # Perplexity is not a coherence score but a measure of how well the model predicts a sample.
    # A lower perplexity indicates better model performance.
    perplexity = model.log_perplexity(corpus)
    print('Perplexity: ', perplexity)

    # Visualize the topics
    vis_data = gensimvis.prepare(model, corpus, gensim_dictionary)
    display(pyLDAvis.display(vis_data))

    # Uncomment the next line if you want to save the plot to a file
    pyLDAvis.save_html(vis_data, 'lda_vis.html')

    from pprint import pprint
    pprint(top_topics)


suggest_topics_using_LDA(train, 'title_nltk')


Average topic coherence: -7.9784.
Coherence Score:  0.3151154865793989
u_mass Coherence Score:  -7.97841917770257
c_npmi Coherence Score:  -0.07637340283075635
Perplexity:  -7.419878123352853


[([(0.09182623, 'error'),
   (0.03609562, 'run'),
   (0.03286876, 'app'),
   (0.032016702, 'window'),
   (0.028711878, 'fail'),
   (0.027719067, 'code'),
   (0.025623444, 'studio'),
   (0.025609087, 'server'),
   (0.024997085, 'android'),
   (0.017258262, 'package'),
   (0.017229417, 'support'),
   (0.016059162, 'php'),
   (0.015876967, 'install'),
   (0.014698441, 'connection'),
   (0.014317982, 'script'),
   (0.014094239, 'user'),
   (0.01239085, 'message'),
   (0.012045734, 'instal'),
   (0.010363376, 'authentication'),
   (0.010095769, 'debug')],
  -5.1704995658294015),
 ([(0.07083661, 'get'),
   (0.059045754, 'python'),
   (0.04589399, 'string'),
   (0.044533268, 'value'),
   (0.03295485, 'array'),
   (0.026850455, 'list'),
   (0.026824625, 'convert'),
   (0.026115004, 'way'),
   (0.023318028, 'column'),
   (0.021251487, 'access'),
   (0.018807737, 'panda'),
   (0.018488497, 'check'),
   (0.015969247, 'dataframe'),
   (0.015590256, 'remove'),
   (0.015548599, 'key'),
   (0.0151550

In [ ]:
suggest_topics_using_LDA(train, 'title_spacy')


Average topic coherence: -10.1811.
Coherence Score:  0.34107572737302855
u_mass Coherence Score:  -10.181116540396106
c_npmi Coherence Score:  -0.16394253626354924
Perplexity:  -7.244251882553048
[([(0.1088843, 'file'),
   (0.09735342, 'error'),
   (0.046305217, 'create'),
   (0.041523013, 'object'),
   (0.038127135, 'fail'),
   (0.037298612, 'app'),
   (0.032167308, 'request'),
   (0.020183451, 'access'),
   (0.0201459, 'custom'),
   (0.019307503, 'database'),
   (0.016948026, 'header'),
   (0.016489826, 'connection'),
   (0.01331621, 'open'),
   (0.013073031, 'fix'),
   (0.013022742, 'attribute'),
   (0.0118648, 'issue'),
   (0.0118203955, 'model'),
   (0.01159083, 'json'),
   (0.010712345, 'directory'),
   (0.01068989, 'response')],
  -6.6796997619582354),
 ([(0.07121483, 'run'),
   (0.05644096, 'test'),
   (0.032027528, 'user'),
   (0.031868793, 'pass'),
   (0.031521995, 'form'),
   (0.03025191, 'query'),
   (0.02957595, 'parameter'),
   (0.027431587, 'url'),
   (0.023802273, 'inpu

In [ ]:
suggest_topics_using_LDA(train, 'body_nltk')


Average topic coherence: -1.9416.
Coherence Score:  0.6894033505037858
u_mass Coherence Score:  -1.9415548758556298
c_npmi Coherence Score:  0.06679520494081188
Perplexity:  -7.227426799941374
[([(0.022210108, 'http'),
   (0.022037039, 'request'),
   (0.01924213, 'server'),
   (0.015685705, 'api'),
   (0.013877183, 'return'),
   (0.013840087, 'client'),
   (0.01323554, 'user'),
   (0.0131452, 'response'),
   (0.013115957, 'send'),
   (0.013103471, 'log'),
   (0.012728417, 'post'),
   (0.011579181, 'json'),
   (0.01136715, 'access'),
   (0.010194902, 'get'),
   (0.010072962, 'com'),
   (0.009759763, 'application'),
   (0.009750408, 'data'),
   (0.009410587, 'web'),
   (0.0094046835, 'header'),
   (0.009173275, 'error')],
  -1.4819711437941332),
 ([(0.02160969, 'want'),
   (0.02144259, 'work'),
   (0.021353683, 'way'),
   (0.019474566, 'need'),
   (0.016569873, 'make'),
   (0.015642902, 'know'),
   (0.015454985, 'code'),
   (0.01516183, 'find'),
   (0.01438039, 'try'),
   (0.014132317, '

In [ ]:
suggest_topics_using_LDA(train, 'body_spacy')


Average topic coherence: -3.3718.
Coherence Score:  0.615722428231344
u_mass Coherence Score:  -3.371750486798459
c_npmi Coherence Score:  0.015584326259350486
Perplexity:  -7.064650275477324
[([(0.031166907, 'use'),
   (0.0256089, 'work'),
   (0.0254038, 'want'),
   (0.022752678, 'way'),
   (0.02133023, 'need'),
   (0.019930327, 'code'),
   (0.017261647, 'know'),
   (0.016318304, 'find'),
   (0.015232047, 'look'),
   (0.014255157, 'example'),
   (0.013543505, 'try'),
   (0.013448542, 'create'),
   (0.013280336, 'thank'),
   (0.012804566, 'time'),
   (0.012689043, 'question'),
   (0.012505116, 'problem'),
   (0.010774083, 'like'),
   (0.010281604, 'help'),
   (0.009695254, 'solution'),
   (0.009658164, 'change')],
  -1.6694002103159038),
 ([(0.03606861, 'error'),
   (0.035679244, 'file'),
   (0.03020364, 'run'),
   (0.027384477, 'try'),
   (0.017777706, 'project'),
   (0.016713955, 'follow'),
   (0.015824636, 'version'),
   (0.0155252665, 'build'),
   (0.0139276525, 'fail'),
   (0.0130